# DATASET : 

https://www.microsoft.com/en-us/download/details.aspx?id=54765

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,MaxPool2D,Flatten
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import train_test_split

import seaborn as sea
import glob
import cv2
%matplotlib inline

In [ ]:
caminho_dog = glob.glob("PetImages/Dog/*")
caminho_cat = glob.glob("PetImages/Cat/*")

In [ ]:
img_test = plt.imread(caminho_dog[0])
shape = img_test.shape

In [106]:
dogs = np.zeros((1500,128,128,3))
cats = np.zeros((1500,128,128,3))

log = []


i = 0
erro = 0
while i <1500:
    try:
        img = cv2.imread(caminho_dog[i+erro])
        img = cv2.resize(img,(128,128))
        dogs[i] = img/np.amax(img)
        i+=1
        print((i),end="\r")
    except Exception as e:
        log+=[e]
        erro+=1

In [107]:
i = 0
erro = 0
while i <1500:
    try:
        img = cv2.imread(caminho_cat[i+erro])
        img = cv2.resize(img,(128,128))
        cats[i] = img/np.amax(img)
        i+=1
        print((i),end="\r")
    except Exception as e:
        log+=[e]
        erro+=1

In [115]:
y = np.array([0]*1500+[1]*1500)
x = np.concatenate((dogs,cats))

In [116]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

In [117]:
model = Sequential()
model.add(Conv2D(filters=32,kernel_size=(4,4),input_shape=(128,128,3),activation="relu"))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=32,kernel_size=(4,4),input_shape=(32,32,3),activation="relu"))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=32,kernel_size=(2,2),input_shape=(None,None,3),activation="relu"))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Flatten())

model.add(Dense(1024,activation="relu"))

model.add(Dense(512,activation="relu"))

model.add(Dense(256,activation="relu"))

model.add(Dense(1,activation="sigmoid"))

model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])

In [118]:
model.fit(x=X_train,y=y_train,epochs=15,batch_size=300,validation_data=(X_test,y_test))

Epoch 1/15
7/7 [==============================] - 19s 3s/step - loss: 0.7240 - accuracy: 0.4736 - val_loss: 0.6957 - val_accuracy: 0.4869
Epoch 2/15
7/7 [==============================] - 16s 2s/step - loss: 0.6919 - accuracy: 0.5493 - val_loss: 0.6853 - val_accuracy: 0.5141
Epoch 3/15
7/7 [==============================] - 15s 2s/step - loss: 0.6920 - accuracy: 0.4950 - val_loss: 0.6885 - val_accuracy: 0.5131
Epoch 4/15
7/7 [==============================] - 15s 2s/step - loss: 0.6869 - accuracy: 0.5498 - val_loss: 0.6878 - val_accuracy: 0.5131
Epoch 5/15
7/7 [==============================] - 15s 2s/step - loss: 0.6810 - accuracy: 0.5592 - val_loss: 0.6930 - val_accuracy: 0.5131
Epoch 6/15
7/7 [==============================] - 15s 2s/step - loss: 0.6691 - accuracy: 0.5801 - val_loss: 0.6499 - val_accuracy: 0.6434
Epoch 7/15
7/7 [==============================] - 15s 2s/step - loss: 0.6357 - accuracy: 0.6269 - val_loss: 0.6234 - val_accuracy: 0.6636
Epoch 8/15
7/7 [==================

In [128]:
predictions = 1*(model.predict(X_test)>0.5)

In [130]:
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.69      0.68      0.68       508
           1       0.67      0.67      0.67       482

    accuracy                           0.68       990
   macro avg       0.68      0.68      0.68       990
weighted avg       0.68      0.68      0.68       990



In [131]:
confusion_matrix(y_test,predictions)

array([[346, 162],
       [159, 323]], dtype=int64)